In [ ]:
# Deleting the .apperception_cache if it exists, as to avoid DB conflict errors
import os
import shutil

dirpath = os.path.join('.apperception_cache')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

# This piece of code is unsafe, and should not be run if not needed. 
# It serves for test purposes when one recieves a "dead kernel" error.
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
import sys
sys.path.append(os.path.join(os.getcwd(),"apperception"))

### IMPORTS
import lens
import point
from new_world import empty_world

# Let's define some attribute for constructing the world first
name = "trafficScene"  # world name
units = "metrics"  # world units
video_file = "amber_videos/traffic-scene-shorter.mp4"  # example video file
lens_attrs = {"fov": 120, "cam_origin": (0, 0, 0), "skew_factor": 0}
point_attrs = {"p_id": "p1", "cam_id": "cam1", "x": 0, "y": 0, "z": 0, "time": None, "type": "pos"}
camera_attrs = {"ratio": 0.5}
fps = 30

# 1. define a world
traffic_world = empty_world(name)

# 2. construct a camera
fov, res, cam_origin, skew_factor = (
    lens_attrs["fov"],
    [1280, 720],
    lens_attrs["cam_origin"],
    lens_attrs["skew_factor"],
)
cam_lens = lens.PinholeLens(res, cam_origin, fov, skew_factor)

pt_id, cam_id, x, y, z, time, pt_type = (
    point_attrs["p_id"],
    point_attrs["cam_id"],
    point_attrs["x"],
    point_attrs["y"],
    point_attrs["z"],
    point_attrs["time"],
    point_attrs["type"],
)
location = point.Point(pt_id, cam_id, (x, y, z), time, pt_type)

ratio = camera_attrs["ratio"]

# ingest the camera into the world
traffic_world = traffic_world.add_camera(
    cam_id=cam_id,
    location=location,
    ratio=ratio,
    video_file=video_file,
    metadata_identifier=name + "_" + cam_id,
    lens=cam_lens,
)

# Call execute on the world to run the detection algorithm and save the real data to the database
recognized_world = traffic_world.recognize(cam_id)

volume = traffic_world.select_intersection_of_interest_or_use_default(cam_id=cam_id)

In [ ]:
cams = traffic_world.get_camera()
lens = traffic_world.get_len()
# ids = traffic_world.get_id()
print("cameras are", cams)
print("lens are", lens)
# print("ids are", ids)

In [ ]:
# Example Query
filtered_world = recognized_world.filter_traj_type("car").filter_traj_volume(volume).interval(0, fps * 3)
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car").interval(0, fps * 3)
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40))

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car")

## OPTION 1 ###
filtered_world = filtered_world.filter_relative_to_type(relative=lambda obj, camera: -10 <= (camera.x - obj.x) <= 10 \
                                                                                  and 20 <= (camera.y - obj.y) <= 40,
                                                        type="camera")
# The idea is that the user passes in a lambda function, that specifies the relationship that must be met between the queried
# object, and some object of the type passed to the function. In this case, the lambda function filters such that the offset 
# is between -10 and 10 in the x direction, and between 20 and 40 in the y direction, relative to some camera.

### OPTION 2 ###
filtered_world = filtered_world.filter_relative_to_type(offset=((-10, 10), (20, 40), None), heading=None, type="camera")
# The idea is that filter_offset_type() takes in two arguments: the offset in terms of coordinates, a relative heading 
# as well as the type of object to be offset from. In this case, we want it to be somehwere between -10 and 10 units
# offset relative to a camera's x position, somehwere between 20 and 40 units offset relative to some camera's y position, 
# and we dont care about the offset relative to the camera's z position. We also dont care about the relative heading difference.

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car")

## OPTION 1 ###
filtered_world = filtered_world.filter_relative_to_type(relative=lambda obj, camera: -10 <= (camera.x - obj.x) <= 10 \
                                                                                  and 20 <= (camera.y - obj.y) <= 40,
                                                        type="camera")

### OPTION 2 ###
filtered_world = filtered_world.filter_relative_to_type(offset=((-10, 10), (20, 40), None), heading=None, type="camera")

filtered_world = filtered_world.filter_heading(-5, 5)
# Filters for objects that have heading between -5 and 5 degrees

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg relative to ego

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car")

## OPTION 1 ###
filtered_world = filtered_world.filter_relative_to_type(relative=lambda obj, camera: -10 <= (camera.x - obj.x) <= 10 \
                                                                                  and 20 <= (camera.y - obj.y) <= 40 \
                                                                                  and -5 <= (camera.heading - obj.heading) <= 5,
                                                        type="camera")
# Now filtering for a relative heading between -5 and 5 degrees

### OPTION 2 ###
filtered_world = filtered_world.filter_relative_to_type(offset=((-10, 10), (20, 40), None), heading=(-5, 5), type="camera")
# Now filtering for a relative heading between -5 and 5 degrees

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car left of ego by 0.25 

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car")

## OPTION 1 ###
def left_of(obj, camera):
    expec_x = obj.x + 0.25 * np.cos(camera.heading)
    expec_y = obj.y - 0.25 * np.sin(camera.heading)
    # Should also allow some sort of variation, to account for noise (and since exact equality is unlikley)
    return (expec_x == camera.x) and (expec_y == camera.y)

filtered_world = filtered_world.filter_relative_to_type(relative=left_of, type="camera")
# Now filtering such that the car is left of ego by 0.25 units

### OPTION 2 ##
# Not possible

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# badAngle = Range(10, 20) deg
# Car left of ego by 0.25,
# 	facing badAngle relative to ego

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car")

## OPTION 1 ###
def filter(obj, camera):
    expec_x = obj.x + 0.25 * np.cos(camera.heading)
    expec_y = obj.y - 0.25 * np.sin(camera.heading)
    # Should also allow some sort of variation, to account for noise (and since exact equality is unlikley)
    return (expec_x == camera.x) and (expec_y == camera.y) and 10 <= (camera.heading - obj.heading) <= 20

filtered_world = filtered_world.filter_relative_to_type(relative=filter, type="camera")
# Now filtering such that the car is left of ego by 0.25 units

### OPTION 2 ##
# Not possible

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)